# 03 — Spectra Ingest & Relative Reflectance

Convert raw Ocean Insight spectrometer counts into per-scan relative reflectance using the dark and white reference frames. The notebook operates on the subset stored in `data/raw/reflectance/raw_counts/` and writes processed arrays to `data-processed/reflectance/raw_relative/`.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

PROJECT_ROOT = Path.cwd()
RAW_ROOT = PROJECT_ROOT / 'data/raw/reflectance/raw_counts'
RELATIVE_ROOT = PROJECT_ROOT / 'data-processed/reflectance/raw_relative'
RELATIVE_ROOT.mkdir(parents=True, exist_ok=True)

WAVELENGTH_GRID = np.load(PROJECT_ROOT / 'data/reference/reflectance/wavelength_grid.npy')


In [ ]:
def load_counts(path: Path):
    capture = False
    wavelengths = []
    counts = []
    with path.open() as fh:
        for line in fh:
            line = line.strip()
            if line.startswith('>>>>>Begin Spectral Data'):
                capture = True
                continue
            if line.startswith('>>>>>End Spectral Data'):
                break
            if capture and line:
                parts = line.split('	')
                if len(parts) == 2:
                    wl, val = parts
                    wavelengths.append(float(wl))
                    counts.append(float(val))
    return np.array(wavelengths), np.array(counts)


In [ ]:
# Load dark reference (average)
dark_dir = RAW_ROOT / 'Dark_reference'
dark_spectra = []
dark_wavelengths = None
for path in sorted(dark_dir.glob('*.txt')):
    wl, counts = load_counts(path)
    if dark_wavelengths is None:
        dark_wavelengths = wl
    dark_spectra.append(counts)
dark_array = np.vstack(dark_spectra)
dark_mean = dark_array.mean(axis=0)
print(f'Dark frames: {dark_array.shape[0]} scans, {dark_array.shape[1]} wavelengths')


In [ ]:
# Load white reference (average)
white_dir = RAW_ROOT / 'Treatment_5/sample_A/WhiteRef_Calibration_1'
white_spectra = []
for path in sorted(white_dir.glob('*.txt')):
    wl, counts = load_counts(path)
    white_spectra.append(counts)
white_array = np.vstack(white_spectra)
white_mean = white_array.mean(axis=0)
print(f'White frames: {white_array.shape[0]} scans')


In [ ]:
# Sanity check: white tile reflectance relative to itself
reflectance_white = (white_array - dark_mean) / (white_mean - dark_mean)
roi = (dark_wavelengths >= 320) & (dark_wavelengths <= 480)
print('Mean white reflectance in 320-480 nm:', reflectance_white[:, roi].mean())
plt.figure(figsize=(8, 4))
plt.plot(dark_wavelengths, reflectance_white.T, alpha=0.2)
plt.axhline(1.0, color='black', linestyle='--')
plt.title('White reference reflectance (should ~1)')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Relative reflectance')
plt.xlim(300, 650)
plt.show()


In [ ]:
def process_sample(angle: str):
    sample_dir = RAW_ROOT / 'Treatment_5/sample_A' / angle
    spectra = []
    for path in sorted(sample_dir.glob('*.txt')):
        wl, counts = load_counts(path)
        spectra.append(counts)
    sample_array = np.vstack(spectra)
    refl = (sample_array - dark_mean) / (white_mean - dark_mean)
    # Clip to avoid negatives from noise
    refl = np.clip(refl, 0, None)
    out_path = RELATIVE_ROOT / f'Treatment5_sampleA_{angle}.npz'
    np.savez(out_path, wavelength=dark_wavelengths, reflectance=refl)
    print(f'{angle}: {sample_array.shape[0]} scans processed → {out_path.relative_to(PROJECT_ROOT)}')
    return dark_wavelengths, refl

wl_12, refl_12 = process_sample('Angle_12Oclock')
wl_6, refl_6 = process_sample('Angle_6Oclock')


In [ ]:
# Plot a subset of the relative reflectance curves
plt.figure(figsize=(8, 4))
for spectrum in refl_12[:5]:
    plt.plot(wl_12, spectrum, alpha=0.6)
plt.title('Sample A — Treatment 5 (12 o'clock) Relative Reflectance')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Relative reflectance')
plt.xlim(300, 650)
plt.show()


In [ ]:
# Compute simple SNR proxy (mean / std over ROI)
def snr_metric(refl):
    roi = (wl_12 >= 320) & (wl_12 <= 480)
    signal = refl[:, roi].mean(axis=1)
    noise = refl[:, roi].std(axis=1)
    return signal / noise

snr_values = snr_metric(refl_12)
plt.figure(figsize=(6, 4))
plt.hist(snr_values, bins=20)
plt.title('SNR proxy (mean/std) for 12 o'clock scans')
plt.xlabel('SNR proxy')
plt.ylabel('Count')
plt.show()


## Outputs

- Relative reflectance arrays saved as `.npz` archives in `data-processed/reflectance/raw_relative/`.
- White tile sanity check confirms the mean reflectance stays ≈1 across 320–480 nm.
- SNR histogram provides a quick QC snapshot before denoising.

These processed spectra feed the denoising pipeline described elsewhere in the thesis build.